In [6]:
import pandas as pd
from fastai.text import *
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List, Tuple, ClassVar
from fastai.data.block import DataLoaders  # Explicit import for DataLoaders
from fastai.learner import Learner   # Explicit import for Learner

# FastAPI app instance
app = FastAPI()

# Loads user given CSV File
def load_data(file_path):
    # Load only the first 5000 rows for testing
    df = pd.read_csv(file_path, encoding='ISO-8859-1', nrows=5000)
    print(f"Loaded data with {len(df)} records.")
    return df

# Preprocesses data for training
def preprocess_data(df):
    # Make sure the CSV has the correct columns for 'text' and 'sentiment'
    df = df[['text', 'sentiment']]
    
    # Create a DataLoader for text classification
    dls = TextDataLoaders.from_df(df, text_col='text', label_col='sentiment', valid_pct=0.2, bs=32)
    print(f"DataLoader created with {len(dls.train)} training samples and {len(dls.valid)} validation samples.")
    return dls

def learn_data(dls):
    # Define text classifier and fine-tune it
    learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
    learn.fine_tune(4, 1e-2)
    
    return learn

# Define the pydantic models for request and response schemas
class SentimentRequest(BaseModel):
    paragraph: str

class SentimentResponse(BaseModel):
    word_sentiments: List[Tuple[str, str, List[float]]]

# Load the model globally
@app.on_event("startup")
def load_model():
    """ Load the model when the server starts. """
    global dls, learn
    # Load the CSV file containing tweets and sentiments
    file_path = 'train.csv' 
    df = load_data(file_path)
    
    # Preprocess the data and create DataLoader
    dls = preprocess_data(df)

    learn = learn_data(dls)

@app.post("/predict_sentiment/")
def predict_word_sentiment(request: SentimentRequest):
    """ Predict the sentiment for each word in the given paragraph. """
    paragraph = request.paragraph
    
    # Tokenize the paragraph into individual words
    words = paragraph.split()
    
    # Create a list to store sentiment predictions
    word_sentiments = []
    
    # Get predictions for each word
    for word in words:
        # FastAI's `predict` method expects a list of text, so we pass the word as a list
        pred, _, probs = learn.predict(word)
        
        # Append the predicted sentiment and probability for each word
        word_sentiments.append((word, pred, probs.tolist()))  # Convert to list for JSON serialization
    
    return SentimentResponse(word_sentiments=word_sentiments)


/usr/local/lib/python3.10/typing.py:1351: RuntimeWarning: coroutine 'Server.serve' was never awaited
  super().__init_subclass__(*args, **kwargs)
/tmp/ipykernel_22/3635227327.py:44: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
